### 加载数据，切分训练测试集

In [7]:
import pandas as pd

# Read the data
data = pd.read_csv('AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])  
# 需要通过设置true_values，false_values来识别特征找那个的true/false类型
# 特征数据中的True和false，送进模型前，不用处理吗？

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()

Number of rows in the dataset: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


### 由于这是一个小型数据集，我们将使用交叉验证来确保准确测量模型质量

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.980292


根据经验，你会发现找到98％的精确模型是非常罕见的。 可能发生了数据泄露，我们应该更仔细地检查目标泄漏的数据。以下是数据摘要：
* card：如果接受信用卡申请，则为1，否则为0
* reports：主要贬损报告的数量
* age：年龄n岁加上一年的十二分之一
* income：年收入（除以10,000）
* share：每月信用卡支出与年收入的比率 
* expenditure：平均每月信用卡支出
* owner：1如果拥有房屋，0如果租房
* selfempl：如果是自雇人员，则为1，否则为0
* dependents：1 +家属人数
* months：生活在当前地址的月份
* majorcards：持有的主要信用卡数量
* active：活动信用帐户的数量<br/>
这些特征里面，一些变量看起来很可疑。 例如，支出是指支付此卡还是在使用之前使用的卡上的支出？在这个问题上，基础的数据比较可能非常有用

In [3]:
expenditures_cardholders = X.expenditure[y]  # expenditure[y]？
expenditures_noncardholders = X.expenditure[~y] # expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean())) #？
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))  #？

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


如上所示，没有收到卡的每个人都没有支出，而只有2％收到卡的人没有支出。 我们的模型似乎具有高精度并不奇怪。 但这也似乎是目标泄漏的情况，其中支出可能意味着他们申请的卡上的支出。

由于份额部分由支出决定，因此也应予以排除。 变量active和majorcards有点不太清楚，但从描述来看，它们听起来很有意义。 在大多数情况下，如果你无法追踪创建数据的人员以了解更多信息，那么最好是别用。

我们将运行没有目标泄漏的模型，如下所示：

In [4]:
# Drop leaky predictors from dataset
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.831680


这种准确性低了不少，可能令人失望。 但是，我们可以预期在新应用程序中使用它的准确率大约为80％，而泄漏模型可能会比这更糟糕（尽管交叉验证中的表观得分更高）。

### 结论

在许多数据科学应用中，数据泄漏可能是数百万美元的错误。 仔细分离培训和验证数据可以防止训练测试污染，pipeline可以帮助实现这种分离。 同样，谨慎，常识和数据探索的组合可以帮助识别目标泄漏。